In [ ]:
import numpy as np
with open('/content/drive/MyDrive/GenAI_Wksp/Sentiment_Classification/data/reviews.txt', 'r') as f:
  reviews = f.read()

with open('/content/drive/MyDrive/GenAI_Wksp/Sentiment_Classification/data/labels.txt', 'r') as f:
  labels = f.read()

In [ ]:
print(reviews[:1000])
print()
print(labels[:10])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [ ]:
from string import punctuation
reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)
all_words = all_text.split()



In [ ]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label =='positive' else 0 for label in labels_split])

In [ ]:
from collections import Counter

review_lens = Counter([len(x.split()) for x in reviews_split])
print("Zero lenght : {}".format(review_lens[0]))
print("Max lenght : {}".format(max(review_lens)))

Zero lenght : 1
Max lenght : 2514


In [ ]:
non_zero_idx = [ii for ii, review in enumerate(reviews_split) if len(review.split()) != 0]

reviews_split = [reviews_split[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

In [ ]:
from gensim.models import KeyedVectors
embed_lookup = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GenAI_Wksp/Sentiment_Classification/word2vec_model/GoogleNews-vectors-negative300-SLIM.bin',
                                                 binary=True)

In [ ]:
pretrained_words =[]
for index, word in enumerate(embed_lookup.index_to_key):
  pretrained_words.append(word)

In [ ]:
def tokenize_all_reviews(embed_lookup, reviews_split):
    reviews_words = [review.split() for review in reviews_split]

    tokenized_reviews = []
    for review in reviews_words:
        ints = []
        for word in review:
            try:
                idx = embed_lookup.key_to_index[word]
            except:
                idx = 0
            ints.append(idx)
        tokenized_reviews.append(ints)

    return tokenized_reviews



In [ ]:
tokenized_reviews = tokenize_all_reviews(embed_lookup, reviews_split)
print(tokenized_reviews[0])

[0, 137, 3, 0, 11620, 3799, 13, 1215, 10, 9, 194, 54, 12, 73, 61, 685, 41, 183, 243, 129, 12, 1663, 119, 72, 0, 9, 2989, 7334, 242, 159, 0, 453, 2, 0, 137, 1239, 19951, 3, 141, 1980, 0, 1898, 55, 3, 1663, 9, 11124, 0, 3857, 6663, 9, 20401, 295, 28, 45, 148, 157, 102, 27, 15452, 1663, 30714, 9, 65172, 0, 9, 844, 737, 47, 6585, 159, 0, 9, 668, 4365, 1003, 0, 27, 295, 56, 4365, 622, 9, 3832, 0, 43, 0, 897, 3187, 907, 0, 5396, 113, 9, 183, 4365, 1009, 3165, 10, 137, 0, 3288, 296, 10314, 4365, 6638, 213, 0, 8810, 40, 0, 116, 1663, 897, 2059, 0, 0, 137, 4365, 830, 2, 124, 2216, 0, 119, 782, 144, 2, 0, 137, 3, 330, 23046, 78, 0, 16915, 2, 13, 85275, 7451]


In [ ]:
def pad_features(tokenized_reviews, seq_length):
    features = np.zeros((len(tokenized_reviews), seq_length), dtype=int)
    for i, row in enumerate(tokenized_reviews):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features

In [ ]:
sequecne_length = 200
features = pad_features(tokenized_reviews, seq_length=sequecne_length)

print(features[:10,:20])

[[     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [ 16483     26      0     12 106210      0   1698     22     37     24
     432      1     72     30    275      0    303      0    162    126]
 [  1935   1326     12      0   1403     60   3921   2019      3   4809
      36      6   3172   7184    129   7951      0   2180   6098 166268]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0 

In [ ]:
split_frac = 0.8 #80% Training data      20% Test Data
split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]




In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)



In [ ]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SentimentCNN(nn.Module):
    """
    The embedding layer + CNN model that will be used to perform sentiment analysis.
    """

    def __init__(self, embed_model, vocab_size, output_size, embedding_dim,
                 num_filters=100, kernel_sizes=[3, 4, 5], freeze_embeddings=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentCNN, self).__init__()

        self.num_filters = num_filters
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.embedding.weight = nn.Parameter(torch.from_numpy(embed_model.vectors)) # all vectors

        if freeze_embeddings:
            self.embedding.requires_grad = False

        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embedding_dim), padding=(k-2,0))
            for k in kernel_sizes])

        self.fc = nn.Linear(len(kernel_sizes) * num_filters, output_size)

        self.dropout = nn.Dropout(drop_prob)
        self.sig = nn.Sigmoid()


    def conv_and_pool(self, x, conv):

        x = F.relu(conv(x)).squeeze(3)


        x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x_max

    def forward(self, x):
        """
        Defines how a batch of inputs, x, passes through the model layers.
        Returns a single, sigmoid-activated class score as output.
        """
        embeds = self.embedding(x) # (batch_size, seq_length, embedding_dim)
        embeds = embeds.unsqueeze(1)

        conv_results = [self.conv_and_pool(embeds, conv) for conv in self.convs_1d]

        x = torch.cat(conv_results, 1)
        x = self.dropout(x)

        logit = self.fc(x)

        return self.sig(logit)


In [ ]:
vocab_size = len(pretrained_words)
output_size = 1 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 50
kernel_sizes = [3, 4, 5]

net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(net)

SentimentCNN(
  (embedding): Embedding(299567, 300)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 50, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 50, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 50, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=150, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sig): Sigmoid()
)


In [ ]:
lr=0.0001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)



In [ ]:
def train(net, train_loader, epochs, print_every=100):

    if(train_on_gpu):
        net.cuda()

    counter = 0

    net.train()
    for e in range(epochs):

        for inputs, labels in train_loader:
            counter += 1

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            net.zero_grad()

            output = net(inputs)

            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            if counter % print_every == 0:
                val_losses = []
                net.eval()
                for inputs, labels in valid_loader:

                    if(train_on_gpu):
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output = net(inputs)
                    val_loss = criterion(output.squeeze(), labels.float())

                    val_losses.append(val_loss.item())

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
epochs = 5
print_every = 100
train(net, train_loader, epochs, print_every=print_every)

Epoch: 1/5... Step: 100... Loss: 0.686637... Val Loss: 0.685606
Epoch: 1/5... Step: 200... Loss: 0.666189... Val Loss: 0.667301
Epoch: 2/5... Step: 300... Loss: 0.612791... Val Loss: 0.612809
Epoch: 2/5... Step: 400... Loss: 0.493318... Val Loss: 0.534121
Epoch: 3/5... Step: 500... Loss: 0.386140... Val Loss: 0.485331
Epoch: 3/5... Step: 600... Loss: 0.456306... Val Loss: 0.456960
Epoch: 4/5... Step: 700... Loss: 0.381377... Val Loss: 0.434691
Epoch: 4/5... Step: 800... Loss: 0.516579... Val Loss: 0.418425
Epoch: 5/5... Step: 900... Loss: 0.342584... Val Loss: 0.406925
Epoch: 5/5... Step: 1000... Loss: 0.326070... Val Loss: 0.394782


In [ ]:

test_losses = []
num_correct = 0


net.eval()
for inputs, labels in test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    output = net(inputs)

    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


print("Test loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.413
Test accuracy: 0.815


In [ ]:
from string import punctuation

def tokenize_review(embed_lookup, test_review):
    test_review = test_review.lower() # lowercase
    test_text = ''.join([c for c in test_review if c not in punctuation])

    test_words = test_text.split()
    print(test_words)
    tokenized_review = []
    for word in test_words:
        try:
            idx = embed_lookup.key_to_index[word]
        except:
            idx = 0
        tokenized_review.append(idx)

    return tokenized_review


In [ ]:
def predict(embed_lookup, net, test_review, sequence_length=200):
    """
    Predict whether a given test_review has negative or positive sentiment.
    """

    net.eval()

    test_ints = tokenize_review(embed_lookup, test_review)
    print(test_ints)
    seq_length=sequence_length
    features = pad_features([test_ints], seq_length)

    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)

    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()

    output = net(feature_tensor)

    pred = torch.round(output.squeeze())
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))

    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")


In [ ]:
seq_length=200

In [ ]:
test_review_pos = 'worst Movie i have ever seen.'
predict(embed_lookup, net, test_review_pos, seq_length)

['worst', 'movie', 'i', 'have', 'ever', 'seen']
[1398, 1083, 4365, 19, 491, 441]
Prediction value, pre-rounding: 0.012376
Negative review detected.
